In [1]:
%cd /content/drive/MyDrive/
# raw_data is imported from global config

/content/drive/MyDrive


In [2]:
%%capture
! pip install hdf5storage
! pip install mne==0.23.0
! pip install torch
! pip install Braindecode==0.5.1
! pip install timm

***Copy and Paste your code below.***

In [3]:
import os, re
import hdf5storage
import numpy as np
from scipy.io import savemat
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from braindecode.datautil import (create_from_mne_raw, create_from_mne_epochs)
import torch
import timm
import random
from common_dl import set_random_seeds
from common_dl import myDataset
from comm_utils import slide_epochs
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split
from braindecode import EEGClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from braindecode.models import ShallowFBCSPNet,EEGNetv4,Deep4Net
from gesture.models.deepmodel import deepnet,deepnet_rnn,deepnet_changeDepth,deepnet_da
from gesture.models.d2l_resnet import d2lresnet
#from gesture.models.EEGModels import DeepConvNet_210519_512_10
from gesture.models.deepmodel import TSception2

from gesture.myskorch import on_epoch_begin_callback, on_batch_end_callback
from gesture.config import *
from gesture.preprocess.chn_settings import get_channel_setting

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
seed = 20200220  # random seed to make results reproducible
set_random_seeds(seed=seed)

cuda = torch.cuda.is_available()  # check if GPU is available, if True chooses to use it
device = 'cuda' if cuda else 'cpu'
if cuda:
    torch.backends.cudnn.benchmark = True

Attempting to create new mne-python configuration file:
/root/.mne/mne-python.json


In [4]:
import inspect as i
import sys
#sys.stdout.write(i.getsource(deepnet))

In [5]:
if len(sys.argv)>3:
    sid = int(float(sys.argv[1]))
    model_name = sys.argv[2]
    fs = int(float(sys.argv[3]))
    wind = int(float(sys.argv[4]))
    stride = int(float(sys.argv[5]))
    try:
        depth=int(float(sys.argv[6]))
        print("Depth: "+ str(depth))
    except IndexError:
        pass
else: # debug in IDE
    sid=10
    fs=1000
    wind = 500
    stride = 100
    model_name='deepnet_da'
class_number=5

In [6]:
print(stride)

50 41


In [7]:
project_dir=data_dir+'preprocessing'+'/P'+str(sid)+'/'
model_path=project_dir + 'pth' +'/'
result_path=data_dir+'training_result/deepLearning/'+str(sid)+'/'
if not os.path.exists(model_path):
    os.makedirs(model_path)

In [8]:
#[Frequencies[i,1] for i in range(Frequencies.shape[0]) if Frequencies[i,0] == sid][0]

In [9]:
loadPath = data_dir+'preprocessing'+'/P'+str(sid)+'/preprocessing2.mat'
mat=hdf5storage.loadmat(loadPath)
data = mat['Datacell']
channelNum=int(mat['channelNum'][0,0])
data=np.concatenate((data[0,0],data[0,1]),0)
del mat
# standardization
# no effect. why?
if 1==0:
    chn_data=data[:,-4:]
    data=data[:,:-4]
    scaler = StandardScaler()
    scaler.fit(data)
    data=scaler.transform((data))
    data=np.concatenate((data,chn_data),axis=1)

# stim0 is trigger channel, stim1 is trigger position calculated from EMG signal.
chn_names=np.append(["seeg"]*channelNum,["emg0","emg1","stim_trigger","stim_emg"])
chn_types=np.append(["seeg"]*channelNum,["emg","emg","stim","stim"])
info = mne.create_info(ch_names=list(chn_names), ch_types=list(chn_types), sfreq=fs)
raw = mne.io.RawArray(data.transpose(), info)



In [10]:
# gesture/events type: 1,2,3,4,5
events0 = mne.find_events(raw, stim_channel='stim_trigger')
events1 = mne.find_events(raw, stim_channel='stim_emg')
# events number should start from 0: 0,1,2,3,4, instead of 1,2,3,4,5
events0=events0-[0,0,1]
events1=events1-[0,0,1]

#print(events[:5])  # show the first 5
# Epoch from 4s before(idle) until 4s after(movement) stim1.
raw=raw.pick(["seeg"])


In [11]:
len(raw.ch_names)

190

In [12]:
selected_channels=[143, 144, 146, 147, 148, 149, 150, 151, 152, 167]

In [13]:
select=False
if select:
    raw=raw.pick(selected_channels)

In [14]:
# channel selected by selection algorithm

epochs = mne.Epochs(raw, events1, tmin=0, tmax=4,baseline=None)
# or epoch from 0s to 4s which only contain movement data.
# epochs = mne.Epochs(raw, events1, tmin=0, tmax=4,baseline=None)

epoch1=epochs['0'].get_data() # 20 trials. 8001 time points per trial for 8s.
epoch2=epochs['1'].get_data()
epoch3=epochs['2'].get_data()
epoch4=epochs['3'].get_data()
epoch5=epochs['4'].get_data()
list_of_epochs=[epoch1,epoch2,epoch3,epoch4,epoch5]
total_len=list_of_epochs[0].shape[2]

In [15]:
# validate=test=2 trials
trial_number=[list(range(epochi.shape[0])) for epochi in list_of_epochs] #[ [0,1,2,...19],[0,1,2...19],... ]
test_trials=[random.sample(epochi, 2) for epochi in trial_number]
# len(test_trials[0]) # test trials number
trial_number_left=[np.setdiff1d(trial_number[i],test_trials[i]) for i in range(class_number)]

val_trials=[random.sample(list(epochi), 2) for epochi in trial_number_left]
train_trials=[np.setdiff1d(trial_number_left[i],val_trials[i]).tolist() for i in range(class_number)]

# no missing trials
assert [sorted(test_trials[i]+val_trials[i]+train_trials[i]) for i in range(class_number)] == trial_number

test_epochs=[epochi[test_trials[clas],:,:] for clas,epochi in enumerate(list_of_epochs)] # [ epoch0,epoch1,epch2,epoch3,epoch4 ]
val_epochs=[epochi[val_trials[clas],:,:] for clas,epochi in enumerate(list_of_epochs)]
train_epochs=[epochi[train_trials[clas],:,:] for clas,epochi in enumerate(list_of_epochs)]


In [16]:
X_train=[]
y_train=[]
X_val=[]
y_val=[]
X_test=[]
y_test=[]

In [17]:
for clas, epochi in enumerate(test_epochs):
    Xi,y=slide_epochs(epochi,clas,wind, stride)
    assert Xi.shape[0]==len(y)
    X_test.append(Xi)
    y_test.append(y)
X_test=np.concatenate(X_test,axis=0) # (1300, 63, 500)
y_test=np.asarray(y_test)
y_test=np.reshape(y_test,(-1,1)) # (5, 270)

for clas, epochi in enumerate(val_epochs):
    Xi,y=slide_epochs(epochi,clas,wind, stride)
    assert Xi.shape[0]==len(y)
    X_val.append(Xi)
    y_val.append(y)
X_val=np.concatenate(X_val,axis=0) # (1300, 63, 500)
y_val=np.asarray(y_val)
y_val=np.reshape(y_val,(-1,1)) # (5, 270)

for clas, epochi in enumerate(train_epochs):
    Xi,y=slide_epochs(epochi,clas,wind, stride)
    assert Xi.shape[0]==len(y)
    X_train.append(Xi)
    y_train.append(y)
X_train=np.concatenate(X_train,axis=0) # (1300, 63, 500)
y_train=np.asarray(y_train)
y_train=np.reshape(y_train,(-1,1)) # (5, 270)
chn_num=X_train.shape[1]

In [18]:
train_set=myDataset(X_train,y_train)
val_set=myDataset(X_val,y_val)
test_set=myDataset(X_test,y_test)

batch_size = 32
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True, pin_memory=False)
val_loader = DataLoader(dataset=val_set, batch_size=batch_size, shuffle=True, pin_memory=False)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True, pin_memory=False)

train_size=len(train_loader.dataset)
val_size=len(val_loader.dataset)
test_size=len(test_loader.dataset)

In [19]:
# These values we found good for shallow network:
lr = 0.0001
weight_decay = 1e-10
batch_size = 32
n_epochs = 200

In [20]:
one_window=next(iter(train_loader))[0]
n_chans = one_window.shape[1]
input_window_samples=one_window.shape[2]

In [21]:
one_window=next(iter(train_loader))[0]
n_chans = one_window.shape[1]
input_window_samples=one_window.shape[2]

In [22]:

model_name='deepnet'

if model_name=='eegnet':
    #print('Here')
    net = EEGNetv4(n_chans, class_number, input_window_samples=input_window_samples, final_conv_length='auto', drop_prob=0.5)
elif model_name=='shallowFBCSPnet':
    net = ShallowFBCSPNet(n_chans,class_number,input_window_samples=input_window_samples,final_conv_length='auto',) # 51%
elif model_name=='deepnet':
    net = deepnet(n_chans,class_number,wind) # 81%
elif model_name=='deepnet_changeDepth':
    net = deepnet_changeDepth(n_chans,class_number,wind,depth) # 81%
    model_name='deepnet_changeDepth_'+str(depth)
elif model_name == 'deepnet2':
    net = deepnet_seq(n_chans, class_number, wind, )
elif model_name == 'deepnet_rnn':
    net = deepnet_rnn(n_chans, class_number, wind, )  # 65%
elif model_name=='resnet':
    net=d2lresnet() # 92%
elif model_name=='tsception':
    net = TSception2(1000, n_chans, 3, 3, 0.5)
elif model_name=='deepnet_da':
    net = deepnet_da(n_chans, class_number, wind)


if cuda:
    net.cuda()

In [23]:
lr = 0.005
weight_decay = 1e-10

criterion = torch.nn.CrossEntropyLoss()
#criterion = nn.NLLLoss()
#optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9)
optimizer = torch.optim.Adadelta(net.parameters(), lr=lr)
#optimizer = torch.optim.Adam(net.parameters(), lr=lr)
# Decay LR by a factor of 0.1 every 7 epochs
lr_schedulerr = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [24]:

epoch_num = 100
patients=10
train_losses=[]
train_accs=[]
val_accs=[]

In [ ]:
for epoch in range(epoch_num):
    print("------ epoch" + str(epoch) + ": sid"+str(sid)+"@"+model_name+"-----")
    net.train()

    loss_epoch = 0

    running_loss = 0.0
    running_corrects = 0
    for batch, (trainx, trainy) in enumerate(train_loader):
        if isinstance(net, timm.models.visformer.Visformer):
            trainx=torch.unsqueeze(trainx,dim=1)
        optimizer.zero_grad()
        if (cuda):
            trainx = trainx.float().cuda()
        else:
            trainx = trainx.float()
        y_pred = net(trainx)
        #print("y_pred shape: " + str(y_pred.shape))
        preds = y_pred.argmax(dim=1, keepdim=True)
        #_, preds = torch.max(y_pred, 1)

        if cuda:
            loss = criterion(y_pred, trainy.squeeze().cuda().long())
        else:
            loss = criterion(y_pred, trainy.squeeze())

        loss.backward()  # calculate the gradient and store in .grad attribute.
        optimizer.step()
        running_loss += loss.item() * trainx.shape[0]
        running_corrects += torch.sum(preds.cpu().squeeze() == trainy.squeeze())
    #print("train_size: " + str(train_size))
    #lr_schedulerr.step() # test it
    train_loss = running_loss / train_size
    train_acc = (running_corrects.double() / train_size).item()
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    #print("Training loss: {:.2f}; Accuracy: {:.2f}.".format(train_loss,train_acc))
    #print("Training " + str(epoch) + ": loss: " + str(epoch_loss) + "," + "Accuracy: " + str(epoch_acc.item()) + ".")

    running_loss = 0.0
    running_corrects = 0
    if epoch % 1 == 0:
        net.eval()
        # print("Validating...")
        with torch.no_grad():
            for _, (val_x, val_y) in enumerate(val_loader):
                if isinstance(net, timm.models.visformer.Visformer):
                    val_x = torch.unsqueeze(val_x, dim=1)
                if (cuda):
                    val_x = val_x.float().cuda()
                    # val_y = val_y.float().cuda()
                else:
                    val_x = val_x.float()
                    # val_y = val_y.float()
                outputs = net(val_x)
                #_, preds = torch.max(outputs, 1)
                preds = outputs.argmax(dim=1, keepdim=True)

                running_corrects += torch.sum(preds.cpu().squeeze() == val_y.squeeze())

        val_acc = (running_corrects.double() / val_size).item()
        val_accs.append(val_acc)
        print("Training loss:{:.2f},Accuracy:{:.2f}; Evaluation accuracy:{:.2f}.".format(train_loss, train_acc,val_acc))
    if epoch==0:
        best_epoch=0
        best_acc=val_acc
        patient=patients
        state = {
            'net': net.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': epoch,
            # 'loss': epoch_loss
        }
    else:
        if val_acc>best_acc:
            best_epoch=epoch
            best_acc=val_acc
            patient=patients
            state = {
                'net': net.state_dict(),
                'optimizer': optimizer.state_dict(),
                'epoch': epoch,
                #'loss': epoch_loss
            }

        else:
            patient=patient-1
    print("patients left: {:d}".format(patient))
    if patient==0:
        savepath = model_path + 'checkpoint_'+model_name+'_' + str(best_epoch) + '.pth'
        torch.save(state, savepath)

        break


------ epoch0: sid10@deepnet-----
Training loss:1.62,Accuracy:0.21; Evaluation accuracy:0.24.
patients left: 10
------ epoch1: sid10@deepnet-----
Training loss:1.61,Accuracy:0.22; Evaluation accuracy:0.21.
patients left: 9
------ epoch2: sid10@deepnet-----
Training loss:1.60,Accuracy:0.23; Evaluation accuracy:0.22.
patients left: 8
------ epoch3: sid10@deepnet-----
Training loss:1.60,Accuracy:0.24; Evaluation accuracy:0.24.
patients left: 7
------ epoch4: sid10@deepnet-----
Training loss:1.59,Accuracy:0.27; Evaluation accuracy:0.24.
patients left: 10
------ epoch5: sid10@deepnet-----
Training loss:1.58,Accuracy:0.28; Evaluation accuracy:0.26.
patients left: 10
------ epoch6: sid10@deepnet-----
Training loss:1.57,Accuracy:0.30; Evaluation accuracy:0.27.
patients left: 10
------ epoch7: sid10@deepnet-----
Training loss:1.55,Accuracy:0.33; Evaluation accuracy:0.26.
patients left: 9
------ epoch8: sid10@deepnet-----
Training loss:1.52,Accuracy:0.35; Evaluation accuracy:0.28.
patients left:

In [ ]:

checkpoint = torch.load(savepath)
net.load_state_dict(checkpoint['net'])
optimizer.load_state_dict(checkpoint['optimizer'])

net.eval()
# print("Validating...")
with torch.no_grad():
    running_corrects = 0
    for _, (test_x, test_y) in enumerate(test_loader):
        if isinstance(net, timm.models.visformer.Visformer):
            test_x = torch.unsqueeze(test_x, dim=1)
        if (cuda):
            test_x = test_x.float().cuda()
            # val_y = val_y.float().cuda()
        else:
            test_x = test_x.float()
            # val_y = val_y.float()
        outputs = net(test_x)
        #_, preds = torch.max(outputs, 1)
        preds = outputs.argmax(dim=1, keepdim=True)

        running_corrects += torch.sum(preds.cpu().squeeze() == test_y.squeeze())
test_acc = (running_corrects.double() / test_size).item()
print("Test accuracy: {:.2f}.".format(test_acc))

train_result={}
train_result['train_losses']=train_losses
train_result['train_accs']=train_accs
train_result['val_accs']=val_accs
train_result['test_acc']=test_acc

#filename=result_path + 'training_result_'+model_name

filename=result_path + 'training_result_'+model_name+str(sid)+str(wind)
np.save(filename,train_result)

Test accuracy: 0.85.


FileNotFoundError: ignored